In [2]:
import cv2
import mediapipe as mp
import time
import math
import numpy as np

In [4]:
!pip install autopy

  Using cached autopy-4.0.0.tar.gz (20 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      running bdist_wheel
      running build
      running build_py
      running build_ext
      running build_rust
      error: Don't know the correct rust target for system type arm64-apple-darwin20.0.0. Please set the CARGO_BUILD_TARGET environment variable.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for autopy
  Running setup.py clean for autopy
Failed to build autopy
  Running setup.py install for autopy ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for autopy did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      running install
      /Users/arthurspriet/miniforge3/envs/payet/lib/python3.9/site-packages/s

In [3]:
class handDetector():
    def __init__(self, mode=False,modelComplex=1, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.modelComplex = modelComplex
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,self.modelComplex, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
            # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                # print(id, cx, cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax

            if draw:
                cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20), (0, 255, 0), 2)

        return self.lmList, bbox

    def fingersUp(self):
        fingers = []
    # Thumb
        if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] -1][1]:
            fingers.append(1)
        else:
            fingers.append(0)
    # Fingers
        for id in range(1, 5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] -2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
    # totalFingers = fingers.count(1)
        return fingers

    def findDistance(self, p1, p2, img, draw=True, r=15, t=3):
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
            cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
        length = math.hypot(x2 - x1, y2 - y1)

        return length, img, [x1, y1, x2, y2, cx, cy]

def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(0)
    detector = handDetector()
    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmList, bbox = detector.findPosition(img)
        if len(lmList) != 0:
            print(lmList[4])
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 255), 3)
        cv2.imshow("Image", img)
        cv2.waitKey(1)

if __name__ == "__main__":
    main()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[4, 533, 527]
[4, 569, 477]
[4, 609, 457]
[4, 644, 446]
[4, 663, 443]
[4, 649, 432]
[4, 615, 426]
[4, 587, 423]
[4, 561, 423]
[4, 545, 425]
[4, 523, 421]
[4, 521, 420]
[4, 525, 427]
[4, 521, 455]
[4, 523, 452]
[4, 503, 439]
[4, 481, 435]
[4, 475, 448]
[4, 483, 457]
[4, 480, 452]
[4, 481, 469]
[4, 465, 495]
[4, 442, 541]
[4, 352, 693]
[4, 1205, 390]
[4, 1152, 300]
[4, 1155, 296]
[4, 435, 293]
[4, 517, 228]
[4, 605, 218]
[4, 659, 198]
[4, 703, 178]
[4, 739, 177]
[4, 773, 176]
[4, 816, 183]
[4, 838, 190]
[4, 841, 199]
[4, 835, 220]
[4, 845, 255]
[4, 855, 301]
[4, 837, 325]
[4, 820, 367]
[4, 791, 405]
[4, 782, 429]
[4, 784, 445]
[4, 793, 447]
[4, 800, 436]
[4, 786, 419]
[4, 772, 401]
[4, 741, 380]
[4, 733, 361]
[4, 767, 356]
[4, 825, 350]
[4, 868, 356]
[4, 861, 357]
[4, 819, 334]
[4, 780, 331]
[4, 765, 375]
[4, 774, 503]
[4, 1220, 397]
[4, 808, 464]


KeyboardInterrupt: 